In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from itertools import chain
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#torch.cuda.current_device()

# Prepare dataset

In [2]:
PAD_token = 0
SOS_token = 1
EOS_token = 2


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2: "EOS"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je suis en fait plutot flattee .', 'i m actually kind of flattered .']


# Set up model: encoder and decoder

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, device):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.device = device
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    # input shape: (B, seq length)
    # hidden shape: (B, hidden_size)
    # outputs shape: (B, sequence length, hidden_size)
    def forward(self, inputs, hidden):
        embedded = self.embedding(inputs).view(1, -1, self.hidden_size)
        hidden = self.initHidden()
        outputs, last_hidden = self.gru(embedded, hidden)
        return outputs, last_hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, device):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.device=device
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        #self.softmax = nn.LogSoftmax(dim=1)

    # input shape: (B, seq length)
    # embedded shape: (B, seq length, hidden size)
    # outputs shape: (B, seq length, hidden size)
    def forward(self, inputs, hidden):
        embedded = self.embedding(inputs).view(1, -1, self.hidden_size)
        embedded = F.relu(embedded)
        outputs, hidden = self.gru(embedded, hidden)
        outputs = self.out(outputs)
        #output = self.softmax(self.out(output[0]))
        return outputs, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [9]:
# Batch = 1
inputs = torch.tensor([[1,2,3,4,5], [2,3,4,5,6]])
embedding = nn.Embedding(40, 3)
gru = nn.GRU(3, 3, batch_first=True)
out = nn.Linear(3, 6)
embedded = embedding(inputs)
print(embedded.shape)
embedded = F.relu(embedded)
hidden = torch.zeros(1, 1, 3)
outputs, hidden = gru(embedded, hidden)
out(outputs).shape

torch.Size([1, 5, 6])

# Data generator

In [10]:
# dataset for translation
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class dataset(Dataset):
    def __init__(self, device):
        super(dataset, self).__init__()
        self.data = pairs
        self.device = device
    
    def __len__(self):
        return len(pairs)
    
    def __getitem__(self, idx):
        input, output = pairs[idx]
        input_token = torch.tensor([input_lang.word2index[word] for word in input.split()] +
                                   [EOS_token],
                                   device=self.device)
        output_token = torch.tensor([SOS_token] + 
                                    [output_lang.word2index[word] for word in output.split()] +
                                    [EOS_token],
                                    device=self.device)
        return input_token, output_token
    
data = dataset(device)
#dataloader = DataLoader(data,batch_size=2)

# Construct model class

In [14]:
decoder_outputs = torch.tensor([[[0,1,2],[3,4,5]]])
print(decoder_outputs)
print(decoder_outputs[0,:,:])

tensor([[[0, 1, 2],
         [3, 4, 5]]])
tensor([[0, 1, 2],
        [3, 4, 5]])


In [54]:
class seq2seq():
    def __init__(self, input_size, hidden_size, output_size, device):
        super(seq2seq, self).__init__()
        self.device = device
        self.encoder = EncoderRNN(input_size, hidden_size, self.device).to(self.device)
        self.decoder = DecoderRNN(hidden_size,output_size, self.device).to(self.device)
    def train(self, dataset, epochs=5, encoder_lr=0.01, decoder_lr=0.01):
        criterion = nn.CrossEntropyLoss()
        encoder_optimizer = torch.optim.SGD(self.encoder.parameters(), lr=0.01)
        decoder_optimizer = torch.optim.SGD(self.decoder.parameters(), lr=0.01)
        for epoch in range(epochs):
            epoch_loss = 0
            for token_fra, token_eng in dataset:
                # start running encoder
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()
                
                encoder_hidden = self.encoder.initHidden()
                loss = 0
                
                encoder_outputs, encoder_hidden = self.encoder.forward(token_fra,encoder_hidden)

                # encoder ouput is first hidden of decoder
                decoder_hidden = encoder_hidden
                # decoder_outputs shape: (B, seq lengths, hidden size)
                decoder_outputs, decoder_hidden = self.decoder.forward(token_eng, decoder_hidden)
                for index in range(len(token_eng)-1):
                    loss += criterion(decoder_outputs[:,index,:], torch.tensor([token_eng[index+1]]))
                
                epoch_loss += loss
                loss.backward()
                decoder_optimizer.step()
                encoder_optimizer.step()
            print("epoch:", epoch, epoch_loss/len(dataset))
    # data is a encoded sentence like nn.tensor([13,24,56,...])
    def predict(self, inputs):
        with torch.no_grad():
            encoder_hidden = self.encoder.initHidden()
            loss = 0

            encoder_outputs, encoder_hidden = self.encoder.forward(inputs
                                                                ,encoder_hidden)
            decoder_hidden = encoder_hidden
            decoder_output = None
            eng_output = "SOS"
            decoder_token = torch.tensor([SOS_token])
    
            while eng_output != "EOS":
                decoder_output,decoder_hidden = self.decoder.forward(decoder_token, decoder_hidden)
                decoder_token = torch.argmax(decoder_output)
                eng_output = output_lang.index2word[int(decoder_token.numpy())]
                print(eng_output)
    def save(self, model_path):
        encoder_path = model_path + "_encoder"
        decoder_path = model_path + "_decoder"
        torch.save(self.encoder.state_dict(), encoder_path)
        torch.save(self.decoder.state_dict(), decoder_path)
    
    def load(self, model_path):
        self.encoder.load_state_dict(torch.load(model_path + "_encoder"))
        self.decoder.load_state_dict(torch.load(model_path + "_decoder"))

In [47]:
model = seq2seq(input_size=input_lang.n_words, hidden_size=5,output_size=output_lang.n_words, device=device)

In [56]:
model.train(data,epochs=30)

epoch: 0 tensor(16.8882, grad_fn=<DivBackward0>)
epoch: 1 tensor(16.8380, grad_fn=<DivBackward0>)
epoch: 2 tensor(16.8127, grad_fn=<DivBackward0>)
epoch: 3 tensor(16.7763, grad_fn=<DivBackward0>)
epoch: 4 tensor(16.7540, grad_fn=<DivBackward0>)
epoch: 5 tensor(16.7183, grad_fn=<DivBackward0>)
epoch: 6 tensor(16.6828, grad_fn=<DivBackward0>)
epoch: 7 tensor(16.6692, grad_fn=<DivBackward0>)
epoch: 8 tensor(16.6203, grad_fn=<DivBackward0>)
epoch: 9 tensor(16.5786, grad_fn=<DivBackward0>)
epoch: 10 tensor(16.5604, grad_fn=<DivBackward0>)
epoch: 11 tensor(16.5480, grad_fn=<DivBackward0>)
epoch: 12 tensor(16.5310, grad_fn=<DivBackward0>)
epoch: 13 tensor(16.6373, grad_fn=<DivBackward0>)
epoch: 14 tensor(16.5740, grad_fn=<DivBackward0>)
epoch: 15 tensor(16.6237, grad_fn=<DivBackward0>)
epoch: 16 tensor(16.6148, grad_fn=<DivBackward0>)
epoch: 17 tensor(16.6106, grad_fn=<DivBackward0>)
epoch: 18 tensor(16.7421, grad_fn=<DivBackward0>)
epoch: 19 tensor(16.5121, grad_fn=<DivBackward0>)
epoch: 20 

In [57]:
model.save("seq2seq")

# Make prediction

In [58]:
model = seq2seq(input_size=input_lang.n_words, hidden_size=5,output_size=output_lang.n_words, device=device)
model.load("seq2seq")
input_sentence = "je suis mannequin"
input_token = torch.tensor([input_lang.word2index[word] for word in input_sentence.split()] +
                                   [EOS_token],
                                   device=device)
model.predict(input_token)

i
m
not
very
the
the
the
teacher
.
EOS


# seq2seq with attention